In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [109]:
flowlist=pd.read_csv('D:\\ucl\\urbanSimulation\\us\\TubeCommuters_Practical\\flowlist.csv')

In [110]:
flowlist.rename(columns={"station_name":"Orig", "station_name.1": "Dest","orig_out":"Origpop","dest_in":"Destjob",'dis-1':'DIST'}, inplace = True)
flowlist.drop([ 'Unnamed: 0'],  inplace=True, axis = 1)

In [111]:
flowlist

,Orig,Dest,total_wtrips,orig,dest,dis,Origpop,Destjob,DIST
0,Acton Town,Alperton,143,111,238,4785.556297,5910,2961,0.000209
1,Acton Town,Amersham,22,111,172,36052.719260,5910,1384,0.000028
2,Acton Town,Angel,15,111,196,14868.355670,5910,17781,0.000067
3,Acton Town,Archway,9,111,198,17188.348800,5910,7388,0.000058
4,Acton Town,Arsenal,9,111,244,17036.911890,5910,1857,0.000059
...,...,...,...,...,...,...,...,...,...
34053,Woodside Park,Waterloo,93,236,10,15501.293340,4025,87309,0.000065
34054,Woodside Park,West Brompton,0,236,118,19396.508330,4025,4854,0.000052
34055,Woodside Park,West Finchley,34,236,230,953.113159,4025,1374,0.001049
34056,Woodside Park,Westminster,51,236,86,15547.114780,4025,24164,0.000064


In [139]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
log_Dist=np.log(flowlist['dis'])

#create the formula
dbl_form = 'total_wtrips ~ Dest + Orig + log_Dist -1 '

doubSim = smf.glm(formula = dbl_form, data=flowlist, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(doubSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           total_wtrips   No. Observations:                34058
Model:                            GLM   Df Residuals:                    33529
Model Family:                 Poisson   Df Model:                          528
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.6874e+06
Date:                Fri, 23 Apr 2021   Deviance:                   3.2206e+06
Time:                        20:13:18   Pearson chi2:                 4.28e+06
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Dest[Act

In [71]:
flowlist["doubsimfitted"] = np.round(doubSim.predict())
#now we can create pivot table to turn paired list into matrix (and compute the margins as well)
cdatasubmat1 = flowlist.pivot_table(values ="doubsimfitted", index="Orig", columns = "Dest",
                                    aggfunc=np.sum, margins=True)
cdatasubmat1

Dest,Acton Town,Aldgate,Aldgate East,Alperton,Amersham,Angel,Archway,Arnos Grove,Arsenal,Baker Street,...,White City,Whitechapel,Willesden Green,Willesden Junction,Wimbledon,Wimbledon Park,Wood Green,Woodford,Woodside Park,All
Orig,,,,,,,,,,,,,,,,,,,,,
Acton Town,NaN,NaN,NaN,59.0,4.0,30.0,16.0,NaN,5.0,58.0,...,NaN,35.0,16.0,NaN,70.0,15.0,17.0,NaN,NaN,5906.0
Aldgate,17.0,NaN,NaN,NaN,2.0,95.0,36.0,19.0,12.0,96.0,...,NaN,NaN,27.0,NaN,52.0,NaN,45.0,27.0,16.0,9972.0
Aldgate East,18.0,NaN,NaN,NaN,NaN,101.0,39.0,20.0,NaN,102.0,...,41.0,176.0,NaN,NaN,55.0,NaN,NaN,32.0,NaN,11598.0
Alperton,45.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.0,35.0,...,NaN,NaN,NaN,NaN,42.0,NaN,10.0,NaN,NaN,3075.0
Amersham,NaN,6.0,NaN,16.0,NaN,8.0,NaN,NaN,NaN,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2184.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wimbledon Park,NaN,NaN,14.0,10.0,NaN,17.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,151.0,NaN,NaN,NaN,NaN,2506.0
Wood Green,NaN,79.0,70.0,15.0,NaN,108.0,NaN,126.0,32.0,130.0,...,49.0,NaN,36.0,NaN,57.0,NaN,NaN,NaN,NaN,12717.0
Woodford,NaN,50.0,51.0,NaN,NaN,42.0,16.0,NaN,NaN,42.0,...,17.0,87.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,7027.0


In [72]:
cdatasub = flowlist.pivot_table(values ="total_wtrips", index="Orig", columns = "Dest",
                                    aggfunc=np.sum, margins=True)
cdatasub

Dest,Acton Town,Aldgate,Aldgate East,Alperton,Amersham,Angel,Archway,Arnos Grove,Arsenal,Baker Street,...,White City,Whitechapel,Willesden Green,Willesden Junction,Wimbledon,Wimbledon Park,Wood Green,Woodford,Woodside Park,All
Orig,,,,,,,,,,,,,,,,,,,,,
Acton Town,NaN,NaN,NaN,143.0,22.0,15.0,9.0,NaN,9.0,19.0,...,NaN,10.0,0.0,NaN,71.0,97.0,27.0,NaN,NaN,5910
Aldgate,0.0,NaN,NaN,NaN,0.0,46.0,12.0,22.0,0.0,271.0,...,NaN,NaN,18.0,NaN,7.0,NaN,43.0,4.0,0.0,9974
Aldgate East,22.0,NaN,NaN,NaN,NaN,121.0,0.0,0.0,NaN,21.0,...,8.0,592.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,11598
Alperton,192.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.0,6.0,...,NaN,NaN,NaN,NaN,49.0,NaN,0.0,NaN,NaN,3084
Amersham,NaN,52.0,NaN,83.0,NaN,4.0,NaN,NaN,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wimbledon Park,NaN,NaN,24.0,0.0,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,533.0,NaN,NaN,NaN,NaN,2504
Wood Green,NaN,8.0,60.0,27.0,NaN,99.0,NaN,231.0,0.0,35.0,...,45.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,12714
Woodford,NaN,19.0,68.0,NaN,NaN,19.0,0.0,NaN,NaN,36.0,...,44.0,206.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,7024


In [31]:
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [76]:
CalcRMSE(flowlist["total_wtrips"],flowlist["doubsimfitted"])

194.505

In [70]:
beta=doubSim.params[-1]
beta

-9.702684732326743e-05

## Production model

In [112]:
#create the formula (the "-1" indicates no intercept in the regression model).
#log_Dist=np.log(flowlist['dis'])
#log_Dj2_destsal=np.log(flowlist['Dest_job'])
x_variables = ['Destjob', "dis"]
log_x_vars = []
for x in x_variables:
    flowlist[f"log_{x}"] = np.log(flowlist[x])
    log_x_vars.append(f"log_{x}")
formula = 'total_wtrips ~ Orig + log_Destjob + log_dis-1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=flowlist, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim.summary())

#create the formula

                 Generalized Linear Model Regression Results                  
Dep. Variable:           total_wtrips   No. Observations:                34058
Model:                            GLM   Df Residuals:                    33792
Model Family:                 Poisson   Df Model:                          265
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7677e+06
Date:                Fri, 23 Apr 2021   Deviance:                   3.3812e+06
Time:                        19:06:02   Pearson chi2:                 4.94e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Orig[Acton T

In [104]:
alpha_i = prodSim.params[0:-2]
gamma = prodSim.params[-2]
beta = prodSim.params[-1]

In [80]:
beta

-0.4751317197069762

In [113]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(flowlist.groupby(["Orig"])["total_wtrips"].agg(np.sum))
O_i.rename(columns={"total_wtrips":"O_i"}, inplace = True)
flowlist = flowlist.merge(O_i, on = "Orig", how = "left" )

D_j = pd.DataFrame(flowlist.groupby(["Dest"])["total_wtrips"].agg(np.sum))
D_j.rename(columns={"total_wtrips":"D_j"}, inplace = True)
flowlist = flowlist.merge(D_j, on = "Dest", how = "left" )

In [114]:
flowlist

,Orig,Dest,total_wtrips,orig,dest,dis,Origpop,Destjob,DIST,log_Destjob,log_dis,O_i,D_j
0,Acton Town,Alperton,143,111,238,4785.556297,5910,2961,0.000209,7.993282,8.473358,5910,2961
1,Acton Town,Amersham,22,111,172,36052.719260,5910,1384,0.000028,7.232733,10.492738,5910,1384
2,Acton Town,Angel,15,111,196,14868.355670,5910,17781,0.000067,9.785886,9.606990,5910,17781
3,Acton Town,Archway,9,111,198,17188.348800,5910,7388,0.000058,8.907612,9.751987,5910,7388
4,Acton Town,Arsenal,9,111,244,17036.911890,5910,1857,0.000059,7.526718,9.743138,5910,1857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34053,Woodside Park,Waterloo,93,236,10,15501.293340,4025,87309,0.000065,11.377209,9.648679,4025,87309
34054,Woodside Park,West Brompton,0,236,118,19396.508330,4025,4854,0.000052,8.487558,9.872848,4025,4854
34055,Woodside Park,West Finchley,34,236,230,953.113159,4025,1374,0.001049,7.225481,6.859734,4025,1374
34056,Woodside Park,Westminster,51,236,86,15547.114780,4025,24164,0.000064,10.092619,9.651630,4025,24164


In [115]:
#We can do this by pulling out the paramaetr values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(Orig)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
flowlist = flowlist.merge(coefs, left_on="Orig", right_on="coef", how = "left")
flowlist.drop(columns = ["coef"], inplace = True)
#check this has worked
flowlist.head()

D:\software\anaconda\envs\urbsim\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,Orig,Dest,total_wtrips,orig,dest,dis,Origpop,Destjob,DIST,log_Destjob,log_dis,O_i,D_j,alpha_i
0,Acton Town,Alperton,143,111,238,4785.556297,5910,2961,0.000209,7.993282,8.473358,5910,2961,1.546872
1,Acton Town,Amersham,22,111,172,36052.719260,5910,1384,0.000028,7.232733,10.492738,5910,1384,1.546872
2,Acton Town,Angel,15,111,196,14868.355670,5910,17781,0.000067,9.785886,9.606990,5910,17781,1.546872
3,Acton Town,Archway,9,111,198,17188.348800,5910,7388,0.000058,8.907612,9.751987,5910,7388,1.546872
4,Acton Town,Arsenal,9,111,244,17036.911890,5910,1857,0.000059,7.526718,9.743138,5910,1857,1.546872


In [116]:
flowlist["prodsimest1"] = np.exp(flowlist["alpha_i"]+gamma*flowlist["log_Destjob"] + beta*flowlist["log_dis"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
flowlist.head(10)

,Orig,Dest,total_wtrips,orig,dest,dis,Origpop,Destjob,DIST,log_Destjob,log_dis,O_i,D_j,alpha_i,prodsimest1
0,Acton Town,Alperton,143,111,238,4785.556297,5910,2961,0.000209,7.993282,8.473358,5910,2961,1.546872,23.418656
1,Acton Town,Amersham,22,111,172,36052.719260,5910,1384,0.000028,7.232733,10.492738,5910,1384,1.546872,5.249462
2,Acton Town,Angel,15,111,196,14868.355670,5910,17781,0.000067,9.785886,9.606990,5910,17781,1.546872,48.332344
3,Acton Town,Archway,9,111,198,17188.348800,5910,7388,0.000058,8.907612,9.751987,5910,7388,1.546872,24.296104
4,Acton Town,Arsenal,9,111,244,17036.911890,5910,1857,0.000059,7.526718,9.743138,5910,1857,1.546872,9.220657
5,Acton Town,Baker Street,19,111,0,11275.334920,5910,26836,0.000089,10.197500,9.330373,5910,26836,1.546872,73.669027
6,Acton Town,Bank,226,111,25,14205.493040,5910,99997,0.000070,11.512895,9.561384,5910,99997,1.546872,166.790933
7,Acton Town,Barbican,7,111,77,15471.033620,5910,13974,0.000065,9.544954,9.646725,5910,13974,1.546872,40.022609
8,Acton Town,Barons Court,36,111,100,4874.227495,5910,6498,0.000205,8.779250,8.491717,5910,6498,1.546872,40.393024
9,Acton Town,Bayswater,2,111,80,8754.398413,5910,2718,0.000114,7.907652,9.077312,5910,2718,1.546872,16.547481


In [117]:
#first round the estimates
flowlist["prodsimest1"] = round(flowlist["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
cdatasubmat3 = flowlist.pivot_table(values ="prodsimest1", index="Orig", columns = "Dest",
                            aggfunc=np.sum, margins=True)
cdatasubmat3

Dest,Acton Town,Aldgate,Aldgate East,Alperton,Amersham,Angel,Archway,Arnos Grove,Arsenal,Baker Street,...,White City,Whitechapel,Willesden Green,Willesden Junction,Wimbledon,Wimbledon Park,Wood Green,Woodford,Woodside Park,All
Orig,,,,,,,,,,,,,,,,,,,,,
Acton Town,NaN,NaN,NaN,23.0,5.0,48.0,24.0,NaN,9.0,74.0,...,NaN,54.0,22.0,NaN,37.0,10.0,26.0,NaN,NaN,5908.0
Aldgate,21.0,NaN,NaN,NaN,5.0,107.0,36.0,17.0,15.0,104.0,...,NaN,NaN,28.0,NaN,37.0,NaN,37.0,21.0,14.0,9977.0
Aldgate East,24.0,NaN,NaN,NaN,NaN,117.0,40.0,19.0,NaN,115.0,...,44.0,257.0,NaN,NaN,41.0,NaN,NaN,25.0,NaN,11602.0
Alperton,31.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,7.0,56.0,...,NaN,NaN,NaN,NaN,29.0,NaN,21.0,NaN,NaN,3083.0
Amersham,NaN,18.0,NaN,8.0,NaN,24.0,NaN,NaN,NaN,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2187.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wimbledon Park,NaN,NaN,22.0,7.0,NaN,26.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,2502.0
Wood Green,NaN,80.0,91.0,20.0,NaN,118.0,NaN,69.0,33.0,149.0,...,61.0,NaN,44.0,NaN,58.0,NaN,NaN,NaN,NaN,12709.0
Woodford,NaN,50.0,60.0,NaN,NaN,62.0,29.0,NaN,NaN,76.0,...,34.0,83.0,NaN,NaN,35.0,NaN,NaN,NaN,NaN,7024.0


In [118]:
CalcRSqaured(flowlist["total_wtrips"], flowlist["prodsimest1"])

0.3741690813864125

In [119]:
CalcRMSE(flowlist["total_wtrips"], flowlist["prodsimest1"])

205.845

### A 'what if...' scenario

In [124]:
def new_sal(row):
    if row["dest"] == 156:
        val = 34058.5
    else:
        val = row["Destjob"]
    return val
        
flowlist["Dj3_destsalScenario"] = flowlist.apply(new_sal, axis =1)

In [131]:
flowlist["prodsimest2"] = np.exp(flowlist["alpha_i"]+gamma*np.log(flowlist["Dj3_destsalScenario"]) + beta*flowlist["log_dis"])

flowlist["prodsimest2"] = round(flowlist["prodsimest2"],0)
#now we can convert the pivot table into a matrix
cdatasubmat4 = flowlist.pivot_table(values ="prodsimest2", index="Orig", columns = "Dest",
                            aggfunc=np.sum, margins=True)
cdatasubmat4['Canary Wharf']

Orig
Acton Town           68.0
Aldgate               NaN
Aldgate East          NaN
Alperton              NaN
Amersham             36.0
                   ...   
Wimbledon Park       37.0
Wood Green          142.0
Woodford            112.0
Woodside Park        63.0
All               29985.0
Name: Canary Wharf, Length: 265, dtype: float64

In [132]:
#calculate some new wj^alpha and d_ij^beta values
Dj2_gamma = flowlist["Destjob"]**gamma
dist_beta = flowlist["dis"]**beta
#calcualte the first stage of the Ai values
flowlist["Ai1"] = Dj2_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(flowlist.groupby(["Orig"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i"}, inplace=True)
#and write the A_i values back into the dataframe
flowlist = flowlist.merge(A_i, left_on="Orig", right_index=True, how="left")

In [133]:
#to check everything works, recreate the original estimates
flowlist["prodsimest3"] = flowlist["A_i"]*flowlist["O_i"]*Dj2_gamma*dist_beta
flowlist["prodsimest3"] = round(flowlist["prodsimest3"])
#check
flowlist[["prodsimest1", "prodsimest3"]]

,prodsimest1,prodsimest3
0,23.0,23.0
1,5.0,5.0
2,48.0,48.0
3,24.0,24.0
4,9.0,9.0
...,...,...
34053,143.0,143.0
34054,17.0,17.0
34055,29.0,29.0
34056,58.0,58.0


In [134]:
#calculate some new wj^alpha and d_ij^beta values
Dj3_gamma = flowlist["Dj3_destsalScenario"]**gamma
#calcualte the first stage of the Ai values
flowlist["Ai1"] = Dj3_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(flowlist.groupby(["Orig"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)
#and write the A_i values back into the dataframe
flowlist = flowlist.merge(A_i, left_on="Orig", right_index=True, how="left")

In [135]:
#to check everything works, recreate the original estimates
flowlist["prodsimest4"] = flowlist["A_i2"]*flowlist["O_i"]*Dj3_gamma*dist_beta
#round
flowlist["prodsimest4"] = round(flowlist["prodsimest4"])

In [136]:
cdatasubmat5 = flowlist.pivot_table(values ="prodsimest4", index="Orig", columns = "Dest",
                            aggfunc=np.sum, margins=True)
cdatasubmat5

Dest,Acton Town,Aldgate,Aldgate East,Alperton,Amersham,Angel,Archway,Arnos Grove,Arsenal,Baker Street,...,White City,Whitechapel,Willesden Green,Willesden Junction,Wimbledon,Wimbledon Park,Wood Green,Woodford,Woodside Park,All
Orig,,,,,,,,,,,,,,,,,,,,,
Acton Town,NaN,NaN,NaN,24.0,5.0,49.0,24.0,NaN,9.0,74.0,...,NaN,54.0,22.0,NaN,38.0,10.0,26.0,NaN,NaN,5911.0
Aldgate,21.0,NaN,NaN,NaN,5.0,107.0,36.0,17.0,15.0,104.0,...,NaN,NaN,28.0,NaN,37.0,NaN,37.0,21.0,14.0,9977.0
Aldgate East,24.0,NaN,NaN,NaN,NaN,117.0,40.0,19.0,NaN,115.0,...,44.0,257.0,NaN,NaN,41.0,NaN,NaN,25.0,NaN,11602.0
Alperton,31.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,7.0,56.0,...,NaN,NaN,NaN,NaN,29.0,NaN,21.0,NaN,NaN,3083.0
Amersham,NaN,18.0,NaN,8.0,NaN,25.0,NaN,NaN,NaN,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2189.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wimbledon Park,NaN,NaN,22.0,7.0,NaN,26.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,61.0,NaN,NaN,NaN,NaN,2503.0
Wood Green,NaN,80.0,92.0,21.0,NaN,119.0,NaN,70.0,33.0,150.0,...,61.0,NaN,45.0,NaN,58.0,NaN,NaN,NaN,NaN,12716.0
Woodford,NaN,51.0,61.0,NaN,NaN,62.0,29.0,NaN,NaN,77.0,...,35.0,84.0,NaN,NaN,35.0,NaN,NaN,NaN,NaN,7021.0


In [137]:

# Here is the entropy maximising approach for a known beta.
# Plug in the required values in this function to solve.

def balance_doubly_constrained(pd, orig_field, dest_field, Oi_field, Dj_field, cij_field, beta, 
                               cost_function, Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = pd[[orig_field, Oi_field]]
    Dj = pd[[dest_field,Dj_field]]    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(pd[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * pd[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    pd[Ai_name] = Oi['Ai_old']
    pd[Bj_name] = Dj['Bj_old']
    return pd

In [140]:
# Use the beta we got from the inverse power model
beta = doubSim.params[-1]
# Get the balancing factors.
flowlist = balance_doubly_constrained(flowlist,'Orig','Dest','O_i','D_j','dis',beta,'power')

# Now predict the model again using the new Ai and Dj fields.
flowlist['SIM_est_pow'] = np.round(flowlist['O_i'] * flowlist['Ai_new'] * flowlist['D_j'] * flowlist['Bj_new'] * 
                                   np.exp(np.log(flowlist['dis'])*beta))
# Check out the matrix
pd.pivot_table(flowlist,values='SIM_est_pow',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


Dest,Acton Town,Aldgate,Aldgate East,Alperton,Amersham,Angel,Archway,Arnos Grove,Arsenal,Baker Street,...,White City,Whitechapel,Willesden Green,Willesden Junction,Wimbledon,Wimbledon Park,Wood Green,Woodford,Woodside Park,All
Orig,,,,,,,,,,,,,,,,,,,,,
Acton Town,0,0,0,47,10,36,22,0,6,56,...,0,45,22,0,61,15,28,0,0,5916.0
Aldgate,19,0,0,0,8,87,32,18,10,78,...,0,0,26,0,54,0,40,25,16,9975.0
Aldgate East,21,0,0,0,0,95,35,20,0,87,...,39,288,0,0,60,0,0,30,0,11596.0
Alperton,35,0,0,0,9,0,0,0,5,41,...,0,0,0,0,46,0,23,0,0,3081.0
Amersham,0,15,0,15,0,19,0,0,0,27,...,0,0,0,0,0,0,0,0,0,2184.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wimbledon Park,0,0,16,12,0,20,0,0,0,0,...,0,0,0,0,135,0,0,0,0,2502.0
Wood Green,0,70,66,34,0,94,0,100,27,117,...,57,0,45,0,91,0,0,0,0,12711.0
Woodford,0,44,44,0,0,47,26,0,0,57,...,32,76,0,0,55,0,0,0,0,7027.0


In [ ]:
# Use the beta we got from the negative exponential model
beta = doubsim1.params[-1]
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
flowlist = balance_doubly_constrained(flowlist,'OrigCodeNew','DestCodeNew','O_i','D_j','Dist',beta,'exponential','Ai_exp','Bj_exp')

# Now predict the model again using the new Ai and Dj fields.
cdatasub['SIM_est_exp'] = np.round(cdatasub['O_i'] * cdatasub['Ai_exp'] * cdatasub['D_j'] * cdatasub['Bj_exp'] * 
                                   np.exp(cdatasub['Dist']*beta))
# Check out the matrix
pd.pivot_table(cdatasub,values='SIM_est_exp',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)